# Sentence Tags Recognition

In [1]:
%matplotlib inline

import pandas as pd

In [2]:
raw_data_set = pd.read_csv("./setence-tags-training-data.csv")

In [3]:
raw_data_set.head(5)

,Sentence type,Sample
0,YES-NO-QUESTION,Can you cancel my order?
1,YES-NO-QUESTION,Could you please create a return?
2,YES-NO-QUESTION,Are you going to cancel it?
3,YES-NO-QUESTION,Is she Dutch?
4,YES-NO-QUESTION,Shall we start?


In [4]:
raw_data_set.describe()

,Sentence type,Sample
count,242,241
unique,22,235
top,RESPONSE ACKNOWLEDGMENT,I see your point
freq,13,2


In [5]:
raw_data_set.columns = ["tag", "text"]
raw_data_set.head()

,tag,text
0,YES-NO-QUESTION,Can you cancel my order?
1,YES-NO-QUESTION,Could you please create a return?
2,YES-NO-QUESTION,Are you going to cancel it?
3,YES-NO-QUESTION,Is she Dutch?
4,YES-NO-QUESTION,Shall we start?


In [6]:
raw_data_set = raw_data_set.dropna()
raw_data_set.describe()

,tag,text
count,241,241
unique,22,235
top,RESPONSE ACKNOWLEDGMENT,I see your point
freq,13,2


In [7]:
raw_data_set["tag"] = pd.Categorical(raw_data_set["tag"])
raw_data_set["tag_code"] = raw_data_set["tag"].cat.codes
raw_data_set.head()

,tag,text,tag_code
0,YES-NO-QUESTION,Can you cancel my order?,21
1,YES-NO-QUESTION,Could you please create a return?,21
2,YES-NO-QUESTION,Are you going to cancel it?,21
3,YES-NO-QUESTION,Is she Dutch?,21
4,YES-NO-QUESTION,Shall we start?,21


In [8]:
raw_data_set["tag_code"].unique()

array([21, 16, 18,  1, 12,  8,  0,  2, 20,  4, 19,  9, 14,  3, 17, 11, 13,
       15,  5, 10,  6,  7])

In [9]:
X = raw_data_set["text"]
y = raw_data_set["tag_code"]

In [10]:
y.describe()

count    241.000000
mean      10.568465
std        6.357777
min        0.000000
25%        5.000000
50%       11.000000
75%       16.000000
max       21.000000
Name: tag_code, dtype: float64

In [14]:
## Importing spacy
import spacy
from sklearn.preprocessing import FunctionTransformer

import contractions

nlp = spacy.load("en_core_web_sm")

ModuleNotFoundError: No module named 'contractions'

In [15]:
def _lemmatize(sentence):
    doc = nlp(sentence)
    lemma_list = [token.lemma_ for token in doc if token.is_alpha]
    return ' '.join(lemma_list)

def lemmatize_transformer(X):
    return X.apply(_lemmatize)

CountVectorizer

In [16]:
_lemmatize("He go")

NameError: name 'nlp' is not defined

In [ ]:
# Model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

In [ ]:
model = make_pipeline(
    SpacyLemmatizer,
    CountVectorizer(), 
    SGDClassifier(loss="log")
)

# Split data
X_train, X_test, y_train, y_text = train_test_split(X, y, test_size=0.2)

# Train
model.fit(X_train, y_train)

# Score
model.score(X_train, y_train)

In [ ]:
sentence = "I don't want that"
sentence_series = pd.Series([_lemmatize(sentence)])

prediction = model.predict(sentence_series)
category = raw_data_set.query("tag_code == " + str(prediction[0]))["tag"].unique().astype(str)[0]

(_lemmatize(sentence), category)